In [ ]:
import requests
import json
import time
import threading
import os
import sys
import csv

import pyquery

In [ ]:
#建立存檔資料夾
def makeDir():
    if not os.path.isdir(f'{filePath}/{brand}'):
        os.mkdir(f'{filePath}/{brand}')
        print(f'mkdir name: {brand}')

#爬蟲 brand_value = 網頁車輛品牌代碼 , code_value = 區域代碼, car_index = 網頁商品編號(從0開始)
def getCarJson(brand_value, code_value, car_index):
    print(f'{brand_value}_{code_value}_{car_index}  start')
    start = time.time()
    Unexpected=None

#取得商品ID_number
    try:
        while True:
            count=0
#發出請求網頁回應，取得商品含有商品ID_number的字串
            while True:
                try:
                    response = requests.get(
                                        url=f'https://www.cargurus.com/Cars/searchResults.action?zip={code_value}&inventorySearchWidgetType=AUTO&nonShippableBaseline=12&hideFreemiumBaseline=false&sourceContext=forSaleTab_false_0&distance=100&entitySelectingHelper.selectedEntity={brand_value}&offset={car_index}&maxResults=25&filtersModified=true'
                                        )
                    if response.status_code != 200:
                        print(f'response status is not 200 ({response.status_code} : {brand_value}_{code_value}_{car_index})')
                        1/0
                    if response.text=='null':
                        1/0
                    break
                except:
                    count+=1
                    time.sleep(1)
#設定失敗20次結束認定此清單已無資料
                    if count>20:
                        break
            
            
#javascript的true/false 轉為python的True/False
            res_str = response.text.replace('true', 'True')
            res_str = response.text.replace('false', 'False')

            if res_str=='null':
                break
            
#將list形式的字串轉為轉為list [ {car1} , {car2}]
#可至底下參考網址 2 查看
            res_list = eval(res_str)

#取得商品ID_number list
            car_id_list = []
            for i in res_list:
                car_id_list.append(i['id'])
                
#以商品ID_number 取得商品資料json檔案
            for car_id in car_id_list:
                response_button = False
                count=0
                while True:
                    try:
                        response = requests.get(f"https://www.cargurus.com/Cars/detailListingJson.action?inventoryListing={car_id}")
#若回應非200，極大可能為商品已下架，直接離開至下一個商品ID_number
                        if response.status_code != 200:
                            print(f'response status is not 200 ({response.status_code} / {car_id})')
                            response_button = True
                            break
                        break
                    except:
                        print(f'try request : {car_id}')
                        count+=1
                        time.sleep(1)
#同一筆資料最多發送20次請求
                        if count>20:
                            response_button = True
                            break
#若response_button為True，則直接至下一個商品ID_number
                if response_button:
                    continue
#所得的回應為json字串
                car_json = response.json()
#存檔用檔案名稱設定（有時會帶贅字故以此處理）
                car_id = str(car_id).split('_')[0]
#存檔以商品ID_number為檔案名稱，避免商品重複
                with open(f'{filePath}/{brand_value}/{car_id}.json', 'w') as json_file:
                    json.dump(car_json, json_file)
                    json_file.close()
                    
#car_index每頁上限25筆，即再搜尋下25筆資料
            car_index+=25
    except:
        Unexpected=sys.exc_info()
    finally:
        Total_time = int(time.time()-start)
#輸出結束訊息
        if Unexpected==None:
            message = f'爬蟲{line_notice_name} / {brand_value}_{code_value}_{car_index} 順利結束，總歷時: {int(Total_time)}'
        else:
            message = f"爬蟲{line_notice_name}中途停止啦! 總歷時: {int(Total_time)}，停止於 : {brand_value}_{code_value}_{car_index}，Unexpected error:{Unexpected}"
        print(message)

        #回傳停止訊息給自己的line
#         token = lineToken
#         requests.post(
#             url='https://notify-api.line.me/api/notify',
#             headers={
#                 'Authorization': f'Bearer {token}'
#             },
#             data={
#                 'message': message
#             }
#         )

# 設定參數

In [ ]:
#存檔資料夾位置
filePath=r'C:\Users\Student\Desktop\car_json'

#line通知時會帶的名稱辨識
line_notice_name = '(iii-1)'
#line權杖（個人設定）
lineToken=''

#網頁車輛品牌代碼
brand = 'm32'


# 讀取zipcode檔案

In [ ]:
with open('data/zipcode.csv', newline='') as csvFile:
    rows = csv.reader(csvFile)
    rows = csv.reader(csvFile, delimiter=',')
    rows = list(rows)
    zip_97884 = rows[0][1:]
    zip_58568 = rows[1][1:]
    zip_60047 = rows[2][1:]
    zip_06371 = rows[3][1:]
    zip_89067 = rows[4][1:]
    zip_79019 = rows[5][1:]
    zip_75935 = rows[6][1:]
    zip_31313 = rows[7][1:]
    csvFile.close()

# 執行爬蟲

In [ ]:
#建立存檔位置
makeDir()

#-------------------------------
#threading.Thread(target = getCarJson , args = (   brand, i , 0 ) )
#target為function, args為傳入function的參數
#以下建立方式為建立多個執行序放進一List中再一次啟動
# i.join()表示該執行序需執行完畢後，才會讓主程式碼繼續往下 (當執行序未中斷前，程式碼會停在.join()等待執行序結束)


#將zipcode分為8個大區塊，每次執行一大區塊
threads = []
for i in zip_97884:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
#-------------------------------
    
threads = []
for i in zip_58568:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    

threads = []
for i in zip_60047:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    
threads = []
for i in zip_06371:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    
threads = []
for i in zip_89067:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    
threads = []
for i in zip_79019:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    
threads = []
for i in zip_75935:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    
threads = []
for i in zip_31313:
    threads.append(threading.Thread(target = getCarJson , args = (   brand, i , 0 )))
for i in threads:
    i.start()
for i in threads:
    i.join()
    

#全部執行完畢會回報一次通知
token = lineToken
requests.post(
    url='https://notify-api.line.me/api/notify',
    headers={
        'Authorization': f'Bearer {token}'
    },
    data={
        'message': f'{line_notice_name}執行緒執行完畢'
    }
)

# 參考網頁

1. 爬蟲的網頁 https://www.cargurus.com
2. 爬蟲中途資料demo https://www.cargurus.com/Cars/searchResults.action?zip=97884&inventorySearchWidgetType=AUTO&nonShippableBaseline=12&hideFreemiumBaseline=false&sourceContext=forSaleTab_false_0&distance=100&entitySelectingHelper.selectedEntity=m1&offset=0&maxResults=25&filtersModified=true
3. 繪製100miles的地圖 https://www.mapdevelopers.com/draw-circle-tool.php
4. 經緯度判斷郵遞區號 https://www.mapdevelopers.com/what-is-my-zip-code.php
5. googlemap
6. 500miles https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B804670%2C44.4791145%2C-118.210773%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C46.0620395%2C-100.9610646%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C42.2036906%2C-88.0738554%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C41.3394923%2C-72.2990718%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C36.6591%2C-114.6656%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C31.7917448%2C-94.1787526%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C35.6232161%2C-102.9738108%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B804670%2C31.7128129%2C-81.0194071%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%5D
7. 100miles https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B804670%2C44.4791145%2C-118.210773%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B160934%2C38.3341138%2C-118.9909681%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C38.8118241%2C-121.5535695%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C40.2022481%2C-122.8986401%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C42.3275255%2C-123.2045381%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C44.6829922%2C-123.0912087%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C46.8999704%2C-123.055262%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C48.5945581%2C-121.2443437%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C48.367009%2C-117.8079761%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C48.1935073%2C-114.168584%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C48.0730382%2C-110.8792762%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C46.3580464%2C-120.1885574%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C46.293287%2C-116.7391291%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C45.9161336%2C-113.2000944%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C46.2880508%2C-109.7868301%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C44.3458336%2C-119.6514865%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C42.672148%2C-120.4204613%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C40.7959672%2C-120.2582846%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C38.3904836%2C-116.610237%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C38.9472128%2C-114.2086196%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C40.1065789%2C-111.7584048%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C42.0527901%2C-110.0728069%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C44.12371%2C-108.9482898%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C44.2161709%2C-111.8903654%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C42.115665%2C-113.389638%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C40.5407109%2C-117.6425442%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C40.9679605%2C-115.2088979%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C42.6492684%2C-117.6680141%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C44.627945%2C-116.8562995%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%2C%5B160934%2C43.6065564%2C-115.1581877%2C%22%23164CAA%22%2C%22%23FFFEED%22%2C0.4%5D%5D